# x402 Facilitator for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

**Deployed on Scaleway Functions:**
- Verify: `https://x402facilitatorjccmtmdr-verify.functions.fnc.fr-par.scw.cloud`
- Settle: `https://x402facilitatorjccmtmdr-settle.functions.fnc.fr-par.scw.cloud`
- Supported: `https://x402facilitatorjccmtmdr-supported.functions.fnc.fr-par.scw.cloud`

In [34]:
import requests
import json
from datetime import datetime

# Facilitator endpoints - Scaleway Functions deployment
FACILITATOR_BASE_URL = "https://x402facilitatorjccmtmdr"
VERIFY_URL = f"{FACILITATOR_BASE_URL}-verify.functions.fnc.fr-par.scw.cloud"
SETTLE_URL = f"{FACILITATOR_BASE_URL}-settle.functions.fnc.fr-par.scw.cloud"
SUPPORTED_URL = f"{FACILITATOR_BASE_URL}-supported.functions.fnc.fr-par.scw.cloud"

# For local testing, uncomment this:
# VERIFY_URL = "http://localhost:8080/verify"
# SETTLE_URL = "http://localhost:8080/settle"
# SUPPORTED_URL = "http://localhost:8080/supported"

print("🚀 x402 Facilitator Endpoints:")
print(f"  Verify: {VERIFY_URL}")
print(f"  Settle: {SETTLE_URL}")
print(f"  Supported: {SUPPORTED_URL}")

🚀 x402 Facilitator Endpoints:
  Verify: https://x402facilitatorjccmtmdr-verify.functions.fnc.fr-par.scw.cloud
  Settle: https://x402facilitatorjccmtmdr-settle.functions.fnc.fr-par.scw.cloud
  Supported: https://x402facilitatorjccmtmdr-supported.functions.fnc.fr-par.scw.cloud


## Test /supported Endpoint

The `/supported` endpoint returns information about which networks, payment schemes, and tokens the facilitator supports.

In [35]:
# Test the /supported endpoint
response = requests.get(SUPPORTED_URL)

print(f"Status Code: {response.status_code}")
supported = response.json()
print(json.dumps(supported, indent=2))

print(f"\n✅ Supported Networks:")
for kind in supported['kinds']:
    print(f"  - {kind['network']} ({kind['scheme']} scheme)")
    if 'assets' in kind:
        for asset in kind['assets']:
            print(f"    • {asset['name']} ({asset['symbol']}): {asset['address']}")

Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10",
      "assets": [
        {
          "address": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6
        },
        {
          "address": "0x94b008aA00579c1307B0EF2c499aD98a8ce58e58",
          "name": "Tether USD",
          "symbol": "USDT",
          "decimals": 6
        }
      ]
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420",
      "assets": [
        {
          "address": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6
        }
      ]
    }
  ],
  "extensions": []
}

✅ Supported Networks:
  - eip155:10 (exact scheme)
    • USDC (USDC): 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85
    • Tether USD (USDT): 0x94b008aA00579c1307B0EF2c499aD98a8ce58e58
  - eip155:111554

In [36]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)
pay_to_address = Web3.to_checksum_address(pay_to_address)
print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# Network and token configuration
CHAIN_ID = 11155420  # Optimism Sepolia
USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60
valid_before = int(datetime.now().timestamp()) + 3600
nonce = "0x" + secrets.token_hex(32)
amount = "10000"  # 0.01 USDC (6 decimals)

# EIP-712 domain for USDC on Optimism Sepolia
# IMPORTANT: The contract's name() returns "USDC", not "USD Coin"!
domain_data = {
    "name": "USDC",
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 types for TransferWithAuthorization
types = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Message data
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(amount),
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": nonce
}

# Create the full EIP-712 message
full_message = {
    "types": types,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
from eth_hash.auto import keccak
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\nEIP-712 Full Hash (keccak256(\\x19\\x01 || domain || message)): {full_eip712_hash.hex()}")
print(f"\nSignature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"Nonce: {nonce}")
print(f"Valid After: {valid_after}")
print(f"Valid Before: {valid_before}")
print(f"\n✅ EIP-712 Signatur erstellt")


Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

EIP-712 Full Hash (keccak256(\x19\x01 || domain || message)): a4efaf63fa6c42531cfd6a36bcebf461a1f248d3fe98d0fc3b61225ae65676e6

Signature: 0x70afb18c328a2c03260077a14e58047dc5d90d6e53631e358919eba53394f7cf574a8f906337fe03a5dca7ed5612038813b0e5e89be894c8c34c1dfe8b719b331c
Signature length: 132 chars (should be 132 with 0x)
Nonce: 0x719e593e188d3e1da92fe09c15d8df621d214dfaabfc47dac26efe4a92cdc759
Valid After: 1766090267
Valid Before: 1766093927

✅ EIP-712 Signatur erstellt


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

In [37]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to Optimism Sepolia
w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")

Prüfe Nonce: 0x719e593e188d3e1da92fe09c15d8df621d214dfaabfc47dac26efe4a92cdc759
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


In [38]:
# Now create a valid x402 payment payload with our real signature
real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": amount,
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

# Send to facilitator (deployed on Scaleway)
response = requests.post(
    VERIFY_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

print(f"Status Code: {response.status_code}")
result = response.json()
print(json.dumps(result, indent=2))

if result.get("isValid"):
    print(f"\n✅ Payment ist GÜLTIG!")
    print(f"Payer: {result.get('payer')}")
else:
    print(f"\n❌ Payment ist UNGÜLTIG")
    print(f"Grund: {result.get('invalidReason')}")

Status Code: 200
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Payment ist GÜLTIG!
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [39]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to Optimism Sepolia to check balance
    w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            print(f"Bridge: https://app.optimism.io/bridge")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print("❌ Konnte nicht mit Optimism Sepolia verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")

Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
ETH Balance: 0.049999747747855487 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [40]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

settle_response = requests.post(
    SETTLE_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

print(f"Status Code: {settle_response.status_code}")
settle_result = settle_response.json()
print(json.dumps(settle_result, indent=2))

if settle_result.get("success"):
    print(f"\n✅ Settlement ERFOLGREICH!")
    print(f"Transaction Hash: {settle_result.get('transaction')}")
    print(f"Network: {settle_result.get('network')}")
    print(f"\nSieh dir die Transaktion an auf:")
    print(f"https://sepolia-optimism.etherscan.io/tx/{settle_result.get('transaction')}")
else:
    print(f"\n❌ Settlement FEHLGESCHLAGEN")
    print(f"Grund: {settle_result.get('errorReason')}")
    if settle_result.get('errorReason') == 'insufficient_funds':
        print(f"\nDer Payer ({settle_result.get('payer')}) hat nicht genug USDC.")
        print(f"Hole dir Test-USDC auf Optimism Sepolia:")
        print(f"1. Bridge von Sepolia ETH → Optimism Sepolia: https://app.optimism.io/bridge")
        print(f"2. Swap ETH → USDC auf Uniswap: https://app.uniswap.org/")

Status Code: 200
{
  "success": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "0x3b346ca81a64d39ef4c029b8910bfc353dbd5f2a51e8d073919d7f8cbea03508",
  "network": "eip155:11155420"
}

✅ Settlement ERFOLGREICH!
Transaction Hash: 0x3b346ca81a64d39ef4c029b8910bfc353dbd5f2a51e8d073919d7f8cbea03508
Network: eip155:11155420

Sieh dir die Transaktion an auf:
https://sepolia-optimism.etherscan.io/tx/0x3b346ca81a64d39ef4c029b8910bfc353dbd5f2a51e8d073919d7f8cbea03508
